In [65]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from nni.algorithms.compression.pytorch.quantization import LsqQuantizer, QAT_Quantizer

import torch.nn as nn
import onnx
import onnx.numpy_helper
from math import ceil
### markus
import numpy as np
import PIL
import os
from re import L
import subprocess
from subprocess import DEVNULL, STDOUT
from xmlrpc.client import boolean
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
#%matplotlib notebook
import myprofC as prof

In [66]:
imgSizeW= 128#1280

imgSizeH= 128#720
gen_img_input_dim_w = imgSizeW
gen_img_input_dim_h = imgSizeH
gen_img_input_channels = 8
#test input image
test_input_data="../convertdemo/dataset/rand_3.jpg"
#Paths
quant_image_path = "../quantization_images"
script_path = "../scripts"
log_path = "../logs"
network_path = "../convertdemo/network"
#Files
perform_script = "perform_r6.sh"
parse_script = "parse_r1.sh"
perform_log_file = "model_execution.log"
parsed_log_file = "model_execution_parsed.log"
model_name="mnist"


# Generate images based on some arbitrary input dimension

In [67]:
path = quant_image_path

def generate_random_images(xdim, ydim, channels=3, count=1, path="."):
    """
    This functions generates random bmp images to use for quantization given
    a defined dimension
        @xdim   .. width of images
        @ydim   .. height of images
        @count  .. number of images
        @path   .. path of images
    """

    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path)
        print("The new directory quantization_images is created!")

    #delete the pre generated bmp/jpg files
    filelist = [ f for f in os.listdir(quant_image_path) if (f.endswith(".jpg") or f.endswith(".bmp") ) ]
    for f in filelist:
        os.remove(os.path.join(quant_image_path, f))
        
    for c in range(count):
        rnd_img = np.random.randint(low=0,high=255, size=(ydim, xdim, channels),dtype=np.uint8) #imag.transpose((1,2,0)
        imag_tp = np.ascontiguousarray(rnd_img, dtype=np.uint8)

        pil_image = PIL.Image.frombytes('RGB',(xdim, ydim), imag_tp)
        pil_image.save(path + "/rand_"+str(c)+".bmp")
        pil_image.save(path + "/rand_"+str(c)+".jpg")



In [68]:

torch.manual_seed(0)
# choose the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## generate 10 random input images based on the provided dimensions
generate_random_images(gen_img_input_dim_w, gen_img_input_dim_h, gen_img_input_channels, count=20, path=quant_image_path)


# Build the normal model

In [69]:
class Mnist(nn.Module):
    def __init__(self, input_channels, out_channels):
        super(Mnist, self).__init__()
        #self.conv1 = nn.Conv2d(num_channels, 20, 5, 1)
        self.conv1 = nn.Conv2d(in_channels = input_channels, out_channels = out_channels, kernel_size = 5, stride = 1)
        # self.conv2 = nn.Conv2d(20, 20, 5, 1)

        # self.conv3 = nn.Conv2d(20, 50, 1, 1)

        # f1 = int(imgSizeX/4) - 3
        # f2 = int(imgSizeY/4) - 3
        # self.fc1 = nn.Linear(f1*f2*50, 500)
        # self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # x = F.max_pool2d(x, 2, 2)
        # x = F.relu(self.conv2(x))

        # x = F.relu(self.conv3(x))

        # x = F.max_pool2d(x, 2, 2)
        # f1 = int(imgSizeX/4) - 3
        # f2 = int(imgSizeY/4) - 3
        # x = x.view(-1, f1*f2*50)
        # x = F.relu(self.fc1(x))
        # x = self.fc2(x)
        return x

In [70]:
class SimpleNN(nn.Module):

    def __init__(self):
        super(SimpleNN, self).__init__()
    
        self.fc1 = nn.Linear(gen_img_input_dim_w*gen_img_input_dim_h, 50)#gen_img_input_dim_w*gen_img_input_dim_h
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # select first dim from [1,C,H,W] ie. [0,:,:,:]
        x = x.select(0,0)
        #print(x.shape)
        # select first channel [0,:,:]
        x = x.select(0,0)
        #print(x.shape)
        x = x.reshape((1,-1))
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = self.fc2(x)
        #print(x.shape)
        #print(x)
        return F.log_softmax(x, dim=1)

In [71]:
#model = SimpleNN()
model = Mnist(gen_img_input_channels, 64)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the model for random images

In [72]:
import glob

def load_images(count=1, path=".", extension='*.bmp'):
    """
    loads images as np arrays; no normalization
    """
    imgs =  []
    files = glob.glob(path + "/" + extension)
    for i, f in enumerate(files):
        img = np.asarray(PIL.Image.open(f), dtype=np.float32).transpose((2,0,1))
        img = img.reshape( (1,img.shape[0],img.shape[1], img.shape[2]) )
        img = torch.from_numpy(img)
        imgs.append(img)

        if i+1 > count:
            break

    return imgs

imgs = load_images(count=2,path=quant_image_path)


In [73]:
# model.train()
# for img in imgs:
#     target = torch.tensor([2])
#     optimizer.zero_grad()
#     output = model(img)
#     #print(output)
#     loss = F.nll_loss(output, target)
#     print(loss.item())
#     #print(target, output)
#     loss.backward()
#     optimizer.step()

In [74]:
loop_run = '10'

[ProfileArray,status] = prof.doProfiling(model,
                 loop_run,
                 gen_img_input_channels,
                 gen_img_input_dim_h,
                 gen_img_input_dim_w,
                 debug=True)

accData=ProfileArray.values.tolist()

------------- Exporting to onnx
Exported graph: graph(%input : Float(1, 8, 128, 128, strides=[131072, 16384, 128, 1], requires_grad=0, device=cpu),
      %conv1.weight : Float(64, 8, 5, 5, strides=[200, 25, 5, 1], requires_grad=1, device=cpu),
      %conv1.bias : Float(64, strides=[1], requires_grad=1, device=cpu)):
  %onnx::Relu_3 : Float(1, 64, 124, 124, strides=[984064, 15376, 124, 1], requires_grad=0, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1], onnx_name="Conv_0"](%input, %conv1.weight, %conv1.bias) # /home/sajjad/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:453:0
  %output : Float(1, 64, 124, 124, strides=[984064, 15376, 124, 1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_1"](%onnx::Relu_3) # /home/sajjad/.local/lib/python3.9/site-packages/torch/nn/functional.py:1457:0
  return (%output)

------------- Checking exported model
------------- Performing the profiling...
>>>>> Setting Environ

vnn_mnist.c: In function ‘vnn_CreateMnist’:
vnn_mnist.c:146:29: warning: unused variable ‘data’ [-Wunused-variable]
  146 |     uint8_t *               data;
      |                             ^~~~
At top level:
vnn_mnist.c:94:17: warning: ‘load_data’ defined but not used [-Wunused-function]
   94 | static uint8_t* load_data
      |                 ^~~~~~~~~


  COMPILE /home/khadas/nbg_unify_mnist/main.c
  COMPILE /home/khadas/nbg_unify_mnist/vnn_post_process.c
make: Nothing to be done for 'all'.
>>>>> Compiling the model on Remote Khadas Done! <<<<<<<
>>>>> Copying test input image to Remote... <<<<<<<
>>>>> Copying test input image to Remote! <<<<<<<
>>>>> Executing the model on Remote Khadas... <<<<<<<
>>>>> Executing the model on Remote Khadas Done! <<<<<<<
>>>>> Executing network on Khadas is saved in model_execution.log <<<<<<<
------------- Performing the profiling done!
Return state run_profiler: 
0
------------- Parsing the profiling results...
Parsing ../logs/model_execution.log....
------------- Parsing the profiling results done!
Return state parse_the_results: 
0
------------- auto_profile done!...


In [75]:
#subprocess.check_call(['../scripts/sajjad.sh'])
#subprocess.check_call(["../scripts/sajjad.sh"])
#!ls -la
#!echo "Hello"
#!bash ../scripts/perform_r6.sh 10  ../convertdemo/dataset/rand_3.jpg ../logs/model_execution.log > ../logs/jupyter.log
#myscript="../scripts/sajjad.sh"
#!bash {myscript}
 

In [76]:
C0 = np.array(ProfileArray[0])
C1 = np.array(ProfileArray[1])
C2 = np.array(ProfileArray[2])
print('---------\n')
print(C0)
print('---------\n')
print(C1)
print('---------\n')
print(C2)
print('---------\n')
print(status)



---------

['Create Neural Network' 'Verify Graph' 'Run the 1 time' 'Run the 2 time'
 'Run the 3 time' 'Run the 4 time' 'Run the 5 time' 'Run the 6 time'
 'Run the 7 time' 'Run the 8 time' 'Run the 9 time' 'Run the 10 time'
 'Total   ' 'Average ']
---------

[' 14658us' ' 677us' ' 680.00us' ' 271.00us' ' 265.00us' ' 268.00us'
 ' 260.00us' ' 265.00us' ' 266.00us' ' 274.00us' ' 294.00us' ' 266.00us'
 ' 3147.00us' ' 314.70us']
---------

[14658.   677.   680.   271.   265.   268.   260.   265.   266.   274.
   294.   266.  3147.   314.]
---------

True


In [77]:
from PIL import Image
from numpy import asarray
image = asarray(Image.open('../convertdemo/dataset/rand_6.jpg'))
image.shape

(128, 128, 3)

In [78]:
image = Image.open('../convertdemo/dataset/rand_6.jpg')

In [79]:
image.size

(128, 128)

In [80]:
from numpy import asarray